In [18]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


### pytorch中的自动求导和反向传播

In [19]:
import torchvision
model = torchvision.models.resnet34(pretrained=True)  # 读取与训练model
data = torch.rand(1, 3, 64, 64) # 随机生成图像数据
labels = torch.rand(1, 1000)    # 随机生成label

In [20]:
prediction = model(data)           # 使用model预测data
print(prediction.shape)

torch.Size([1, 1000])


In [21]:
loss = (prediction - labels).sum()        # 设定loss
print(loss)
loss.backward()          # loss反向传播
print(loss.grad)

tensor(-488.7442, grad_fn=<SumBackward0>)
None


c:\users\lihaiyang\.conda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  after removing the cwd from sys.path.


In [22]:
optim = torch.optim.SGD(model.parameters(), lr=0.01, )     # 创建优化器
optim.step()          # 进行优化，调整模型参数

In [23]:
prediction = model(data)         # 重新预测data
loss = (prediction - labels).sum()
print(loss)

tensor(-4381.4731, grad_fn=<SumBackward0>)


###  探索pytorch中的自动求导过程

#### 1. 创建一个多级的函数调用，初始化器参数

In [32]:
a0 = torch.tensor([1.,2.,3.,], requires_grad=True)
a0.retain_grad()
a1 = a0**2
a1.retain_grad()
b1 = torch.tensor([5.,6.,4.,], requires_grad=True)
b1.retain_grad()
Q = a1**3 - b1**2

#### 2. 梯度的计算
梯度的计算使用链式法则，即使用Q的梯度计算a1， b1梯度，然后计算a0梯度，分别存储在对应的tensor中的`.grad`参数中
因为Q是计算的开始，需要手工传递梯度`[1,1,1]`

In [33]:
Q.backward(torch.ones_like(Q))

In [34]:
print(a1.grad)
print(3*a1**2 == a1.grad)
print(b1.grad)
print(-2*b1 == b1.grad)
print(a0.grad)
print(3*a1**2*2*a0 == a0.grad)

tensor([  3.,  48., 243.])
tensor([True, True, True])
tensor([-10., -12.,  -8.])
tensor([True, True, True])
tensor([   6.,  192., 1458.])
tensor([True, True, True])
